In [2]:
from objects.instance import Instance
from script.affiche import plot_gantt
from script.data import loader 

In [3]:
data = loader('ft06')
inst = Instance(**data)

In [4]:
# Recherche des taches possibles à t
t=0
def recherche_tach(inst,t):
    tache = []
    # Pour chaque jobs
    for j in inst.jobs_list:
        if j.state != "Done":
        # Si la machine de la tâche en cours du job et libre
            if j.current_task.machine.state=="Free":
                # On ajoute cette tache à la liste de tache possible
                tache.append(j.current_task)
            
    # prio
# todo prio
    # Recherche du max parmis les taches possibles
    for k in range(len(tache)):
        maxi = -1
        for e in tache:
            if e.duration > maxi:
                if e.machine.state == "Free":
                    maxi = e.duration
                    maxi_task = e
        
        # assignment
        maxi_task.job.update_current_task(maxi_task.taskID, t)
        try:
            tache.remove(maxi_task)
        except:
            pass
def finish():
    for res in inst.jobs_list:
        if res.state == "Not Done":
            return False
    return True 
        

In [5]:
t = 0
recherche_tach(inst, t)

for time in range(1, 600):
    if finish():
        break;
    print("Temps",time)
    for res in inst.resource_list:
        print(res)
        if res.current_task == -1 :
            recherche_tach(inst, t)
        elif res.current_task.finishDate <= time:
            res.current_task.deallocate_to_ressource("Done")
            recherche_tach(inst, time)

Temps 1
Machine number = 0, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 1, State = Busy, Current Job = 1, Current Task = 0
Machine number = 2, State = Busy, Current Job = 4, Current Task = 0
Machine number = 3, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 4, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 5, State = Free, Current Job = N/A, Current Task = N/A
Temps 2
Machine number = 0, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 1, State = Busy, Current Job = 1, Current Task = 0
Machine number = 2, State = Busy, Current Job = 4, Current Task = 0
Machine number = 3, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 4, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 5, State = Free, Current Job = N/A, Current Task = N/A
Temps 3
Machine number = 0, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 1, State = Busy, Current Job = 1, Curre

In [9]:
plot_gantt(inst, grouping= False)


In [6]:
for j in inst.jobs_list:
    print(j)

Task list
Task # 0 : JobID= 0, taskID= 0, Start= 24, Finish=25, Machine= 2, State= Done
Task # 1 : JobID= 0, taskID= 1, Start= 25, Finish=28, Machine= 0, State= Done
Task # 2 : JobID= 0, taskID= 2, Start= 28, Finish=34, Machine= 1, State= Done
Task # 3 : JobID= 0, taskID= 3, Start= 34, Finish=41, Machine= 3, State= Done
Task # 4 : JobID= 0, taskID= 4, Start= 62, Finish=65, Machine= 5, State= Done
Task # 5 : JobID= 0, taskID= 5, Start= 65, Finish=71, Machine= 4, State= Done
Job state= Done
Task list
Task # 0 : JobID= 1, taskID= 0, Start= 0, Finish=8, Machine= 1, State= Done
Task # 1 : JobID= 1, taskID= 1, Start= 9, Finish=14, Machine= 2, State= Done
Task # 2 : JobID= 1, taskID= 2, Start= 14, Finish=24, Machine= 4, State= Done
Task # 3 : JobID= 1, taskID= 3, Start= 31, Finish=41, Machine= 5, State= Done
Task # 4 : JobID= 1, taskID= 4, Start= 41, Finish=51, Machine= 0, State= Done
Task # 5 : JobID= 1, taskID= 5, Start= 51, Finish=55, Machine= 3, State= Done
Job state= Done
Task list
Task 